# Ejercicio 2 - Adrián José Zapater Reig

### Inicialización

In [1]:
! docker run --name mongo_db -it -p 27017:27017 -d mongo:4.0

98d8db67f9b53fd83a97684d3b4daf82c979b29b2799a56086681cd4a5819d0a


In [2]:
! docker cp src/webs.json mongo_db:/tmp/webs.json

In [3]:
! docker exec -it mongo_db mongoimport --db ejercicio2 --collection webs --file /tmp/webs.json

2021-04-11T20:21:16.840+0000	connected to: localhost
2021-04-11T20:21:16.862+0000	imported 51 documents


In [4]:
import pymongo
from pymongo import MongoClient
import pprint
from datetime import datetime

client = MongoClient("localhost", 27017) 
db = client.ejercicio2

In [5]:
# Comprobamos la conexión:
db.list_collection_names()

['webs']

In [6]:
# Mostramos un documento:
db.webs.find_one()

{'_id': ObjectId('60735a3c6ea299541e5d2146'),
 'nombre': 'Bertoldo',
 'navegador': 'firefox',
 'fecha': datetime.datetime(2016, 2, 21, 6, 1, 17, 171000),
 'pags': [{'url': 'http://www.ucm.es', 'segs': 200},
  {'url': 'https://www.todomesobra.com', 'segs': 5},
  {'url': 'https://www.mikokoloko.com', 'segs': 120},
  {'url': 'http://www.mundoroedor.es', 'segs': 85},
  {'url': 'https://www.mikokoloko.com', 'segs': 600}]}

In [7]:
# Función auxiliar para imprimir por pantalla los resultados:
def print_query(query_cursor):
    for doc in query_cursor:
        pprint.pprint(doc)

## Consulta 1
Número de sesiones (documentos) en los que se ha usado el navegador Explorer.

In [8]:
find_json_statement = {"navegador": "Explorer"}

In [9]:
# Nomenglatura como en la shell.
# Deprecado.
db.webs.find(find_json_statement).count()

<ipython-input-9-1aef0dccc92f>:3: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  db.webs.find(find_json_statement).count()


20

In [10]:
db.webs.count_documents(find_json_statement)

20

## Consulta 2
Mostrar las URLs de las páginas webs que se han consultado.

In [11]:
key = "pags.url"
db.webs.find().distinct(key)

['http://www.bancoimpopular.com',
 'http://www.cocinacochina.com',
 'http://www.comomemongo.com',
 'http://www.kalvossinreparos.es',
 'http://www.mikokoloko.com',
 'http://www.mundoroedor.es',
 'http://www.nocheviejuna.es',
 'http://www.nomiresatras.es',
 'http://www.quizaseahoy.es',
 'http://www.ucm.es',
 'http://www.uvasexpress.es',
 'http://www.vuelosparaAbuelos.es',
 'https://www.lanochemashermosa.es',
 'https://www.mikokoloko.com',
 'https://www.secretosindiscretos.com',
 'https://www.todomesobra.com',
 'https://www.ucm.com']

## Consulta 3
Mostrar únicamente el campo navegador empleado por Herminia, cuyas en las sesiones que tengan visitas a páginas webs que hayan durado entre 300 segundos y 1000 segundos.

In [12]:
fields = ["navegador"]
json_find_statement = { 
    "nombre": "Herminia", 
    "pags.segs" : {'$gte': 300, '$lte': 1000}
}

query_cursor = db.webs.find(filter=json_find_statement, projection=fields)
print_query(query_cursor)

{'_id': ObjectId('60735a3c6ea299541e5d2147'), 'navegador': 'Explorer'}
{'_id': ObjectId('60735a3c6ea299541e5d2149'), 'navegador': 'Explorer'}
{'_id': ObjectId('60735a3c6ea299541e5d214d'), 'navegador': 'Explorer'}
{'_id': ObjectId('60735a3c6ea299541e5d214f'), 'navegador': 'Explorer'}
{'_id': ObjectId('60735a3c6ea299541e5d2153'), 'navegador': 'Explorer'}
{'_id': ObjectId('60735a3c6ea299541e5d2155'), 'navegador': 'Explorer'}
{'_id': ObjectId('60735a3c6ea299541e5d2159'), 'navegador': 'Explorer'}
{'_id': ObjectId('60735a3c6ea299541e5d215b'), 'navegador': 'Explorer'}
{'_id': ObjectId('60735a3c6ea299541e5d215f'), 'navegador': 'Explorer'}
{'_id': ObjectId('60735a3c6ea299541e5d2163'), 'navegador': 'Explorer'}
{'_id': ObjectId('60735a3c6ea299541e5d2165'), 'navegador': 'Explorer'}
{'_id': ObjectId('60735a3c6ea299541e5d2169'), 'navegador': 'Explorer'}
{'_id': ObjectId('60735a3c6ea299541e5d216b'), 'navegador': 'Explorer'}
{'_id': ObjectId('60735a3c6ea299541e5d216e'), 'navegador': 'Explorer'}
{'_id'

## Agregación 1
Obtener el número de sesiones por cada navegador, ordenadas en orden ascendente. La salida mostrará, para cada navegador, un “_id” que contiene el nombre del navegador, y un campo "total" con el número de sesiones de dicho navegador.

In [13]:
json_aggregate_statement = [
    {
        "$group": {
            "_id" : "$navegador",
            "total": {
                "$sum" : 1
            }
        }
        
    },
    {
        "$sort": {
            "total": 1
        }
    }
]
query_cursor = db.webs.aggregate(json_aggregate_statement)
print_query(query_cursor)

{'_id': 'Chrome', 'total': 14}
{'_id': 'firefox', 'total': 17}
{'_id': 'Explorer', 'total': 20}


## Agregación 2
Obtener la persona con mayor número de sesiones totales en la colección. La salida mostrará para cada persona (sólo 1) un “_id” conteniendo el nombre de la persona y un campo “total” con el número de sesiones totales.

In [14]:
json_aggregate_statement = [
    {
        "$group": {
            "_id" : "$nombre",
            "total": {
                "$sum" : 1
            }
        }
        
    },
    {
        "$sort": {
            "total": -1
        }
    },
    {
        "$limit" : 1
    }
]
query_cursor = db.webs.aggregate(json_aggregate_statement)
print_query(query_cursor)

{'_id': 'Herminia', 'total': 20}


## Agregación 3
Para cada persona indicar el número de URLs visitadas durante más de 100 segundos, contando las repeticiones. La salida mostrará para cada persona un “_id” conteniendo el nombre de la persona y un campo “num” con el número de URLs. Pista: utilizar la opción unwind para devolver la lista de pags como documentos independientes.

In [15]:
json_aggregate_statement = [
    {
        "$unwind": "$pags"
    },
    {
        "$match": { "pags.segs" : { "$gt" : 100}}
    }, 
    {
        "$group": {
            "_id" : "$nombre",
            "num" : { "$sum": 1}
        }
    }
]
query_cursor = db.webs.aggregate(json_aggregate_statement)
print_query(query_cursor)

{'_id': 'Aniceto', 'num': 35}
{'_id': 'Herminia', 'num': 40}
{'_id': 'Bertoldo', 'num': 51}


## Agregación 4
Mostrar por cada URL, el tiempo de visita total, ordenando los registros de mayor a menor. Pista: utilizar la opción $unwind de aggregate.

In [16]:
json_aggregate_statement = [
    {
        "$unwind": "$pags"
    }, 
    {
        "$group": {
            "_id" : "$pags.url",
            "num" : { "$sum": "$pags.segs"}
        }
    },
    {
        "$sort": {
            "num": -1
        }
    }
]
query_cursor = db.webs.aggregate(json_aggregate_statement)
print_query(query_cursor)

{'_id': 'http://www.nomiresatras.es', 'num': 12000}
{'_id': 'http://www.nocheviejuna.es', 'num': 12000}
{'_id': 'http://www.quizaseahoy.es', 'num': 8400}
{'_id': 'https://www.ucm.com', 'num': 7456}
{'_id': 'https://www.mikokoloko.com', 'num': 6240}
{'_id': 'http://www.vuelosparaAbuelos.es', 'num': 5000}
{'_id': 'http://www.bancoimpopular.com', 'num': 5000}
{'_id': 'http://www.comomemongo.com', 'num': 3145}
{'_id': 'http://www.cocinacochina.com', 'num': 3000}
{'_id': 'http://www.kalvossinreparos.es', 'num': 2100}
{'_id': 'http://www.ucm.es', 'num': 1400}
{'_id': 'https://www.secretosindiscretos.com', 'num': 840}
{'_id': 'http://www.mikokoloko.com', 'num': 600}
{'_id': 'http://www.mundoroedor.es', 'num': 595}
{'_id': 'http://www.uvasexpress.es', 'num': 500}
{'_id': 'https://www.lanochemashermosa.es', 'num': 350}
{'_id': 'https://www.todomesobra.com', 'num': 35}


## Agregación 5
Calcular el total de sesiones con el navegador “Explorer” en 2016. La salida mostrará el “_id” conteniendo el año (2016) y un campo “total” con el número total de sesiones. Pista: para obtener el año de una fecha se puede utilizar \{\\$year:”\\$fecha”}.

In [17]:
json_aggregate_statement = [
    {
        "$project": {
            "navegador" : 1,
            "year" : {"$year":"$fecha"}
        }
    },
    {
        "$match" : { 
            "navegador" : "Explorer" , 
             "year" : 2016
        }
    },
    {
        "$group" : {
            "_id" : "2016",
            "total" : {"$sum" : 1}
        }
    }
]
query_cursor = db.webs.aggregate(json_aggregate_statement)
print_query(query_cursor)

{'_id': '2016', 'total': 10}


## Agregación 6
Para cada persona calcular el tiempo acumulado de todas las URLs visitadas, en minutos. La salida mostrará un campo “nombre” con el nombre de la persona, y un campo “tiempomin” con el tiempo acumulado. La salida estará ordenada alfabéticamente por los nombres de los usuarios. Pista: utilizar la opción \\$unwind de aggregate.


In [18]:
json_aggregate_statement = [
    {
        "$unwind": "$pags"
    },
    {
        "$group" : {
            "_id" : "$nombre",
            "tiempomin" : { "$sum": {"$multiply" : ["$pags.segs", 60]}}
        }
    },
    {
        "$project": {
            "nombre" : "$_id",
            "_id" : 0,
            "tiempomin" : 1
        }
    },
    {
        "$sort": { "nombre" : 1}
    }
]
query_cursor = db.webs.aggregate(json_aggregate_statement)
print_query(query_cursor)

{'nombre': 'Aniceto', 'tiempomin': 1185660}
{'nombre': 'Bertoldo', 'tiempomin': 948000}
{'nombre': 'Herminia', 'tiempomin': 1986000}


### Breakdown

In [19]:
! docker stop mongo_db

mongo_db


In [20]:
! docker rm mongo_db

mongo_db
